# Pandas - a quick introduction

In this tutorial, we'll explore pandas, the Python data analysis toolkit. Our emphasis will be on working with the two primary pandas objects: Series and DataFrame. Although we don't have time to study all the features of pandas, extensive documentation can be found here https://pandas.pydata.org/pandas-docs/stable/dsintro.html

+ Series: one-dimensional labeled array capable of holding any data type
+ DataFrame: two-dimensional labeled data structure with columns of potentially different types.

Let's dive into an Olympic Medal dataset available from Wikipedia https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table. I've already done a little bit of cleanup so that we can quickly get to the important features.

In [ ]:
import pandas as pd

Our file is in csv format, so we'll use the read_csv method. We know that the first two rows contain comments and other data that we won't want. We can use the skiprows argument to skip over these rows. We'll set index_col to zero so that the first column serves as the index. After loading the DataFrame, we'll look at the first and last few rows using the head and tail methods

In [ ]:
df = pd.read_csv('olympics.csv', index_col=0, skiprows=2)

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

Note that the last row of our dataframe contains totals for the number of games and medals won. Let's get rid of that using the drop method.

In [ ]:
df = df.drop('Totals')
df.tail(3)

We can use the shape and size attributes of the dataframe to determine the dimensions and number of cells. Note that the dimensions do not include the column headers and index column.

In [ ]:
df.shape

In [ ]:
df.size

The columns attribute returns the column names

In [ ]:
df.columns

The row labels are returned using the index attribute

In [ ]:
df.index

We can manipulate the data in our data frame. For example, let's cleanup the country names to get rid of everything from the country abbreviation to the end of the string. For example "Australia (AUS) [AUS] [Z]" becomes simply "Australia".

We'll start by calling the str.split method to split the country names on the white space and opening parenthesis "(" and then reassign the first element of the resulting list to the index. We then display the head of the dataframe to confirm that the renaming of the countries worked as expected.

In [ ]:
names_ids = df.index.str.split('\s\(')
df.index = names_ids.str[0]
df.head(3)

## Single and double square brackets / accessing rows and columns

To work with DataFrames, we just need to master a few key concepts

+ Single and double brackets
  * Single brackets [ ] with a single value return a new Series
  * Single brackets [ ] with multiple values return a new DataFrame
  * Double brackets [[ ]] return a new DataFrame


+ Selecting data by row
  * Rows are accessed by position using iloc
  * Rows are accessed by row name or label using loc  


+ Selecting data by column
  * Columns are accessed by column name

### Selecting rows

In [ ]:
# Return a Series selecting row 2 (iloc and single brackets)
df.iloc[2].head()

In [ ]:
# Return a DataFrame selecting row 2-6 (iloc and single brackets)
df.iloc[2:5]

In [ ]:
# Return a single-row Data Frame selecting row 2 (iloc and double brackets)
df.iloc[[2]]

We can also access a row by the label or value of the index using the loc method. For example, if we wanted to get the data for France, it's more convenient to use the label than figure out the row number.

In [ ]:
# Return a Series for row labeled France (loc and single brackets)
df.loc['France'].head()

In [ ]:
# Return a DataFrame for rows labeled France and Germany (loc, single brackets, list argument)
countries = ['France', 'Germany']
df.loc[countries]

In [ ]:
# Return a single-row DataFrame for row labeled France
df.loc[['France']]

### Selecting columns

In [ ]:
# Return column as a Series (column name and single brackets)
df['Summer silver'].head()

In [ ]:
# Return column as a DataFrame (column name and double brackets)
df[['Summer silver']].head()

We can select multiple columns from a dataframe by passing a list of column names rather than a single name. (A data frame is returned when using the double bracket operator)

In [ ]:
# Return multiple columns as DataFrame (list of column names and single brackets)
cnames = ['Summer gold', 'Summer silver', 'Summer bronze']
df[cnames].head()

### Adding and deleting columns

A new column is created simply by assigning data to a column that does not already exist. In the example below, we create a new column that is a weighted sum of all gold, silver and bronze medals. Note that we're using single brackets and operating on series

In [ ]:
df['Combined weighted'] = df['Combined gold']*3 + df['Combined silver']*2 + df['Combined bronze']
df.head()

To delete a column, use the del operator or use the drop method with axis 1

In [ ]:
# This will also work: "df.drop('Combined weighted', 1)"

del df['Combined weighted']
df.head()

## Putting it all together

In this section we show how to find min and max values of columns and the indexes of the corresponding rows. We also show how to filter by column values and combine slicing by row and column. We start by finding the country that won te most winter gold medals and the number they won.

In [ ]:
df['Winter gold'].max()

In [ ]:
df['Winter gold'].idxmax()

We can select rows from the dataframe based on the values in a column. In the example below, we filter on countries that have won at least 50 gold medals in the winter Olympics.

In [ ]:
df.loc[ df['Winter gold']>50 ]

We can filter on multiple columns. In the example below, we limit the output to countries that also won more than 50 summer gold medals and assign the results to a new dataframe

In [ ]:
df2 = df.loc[ (df['Winter gold']>50) & (df['Summer gold']>50) ]
df2

Let's create a simpler dataframe that is limited to the gold medal results and then use the sum method to sum the values in a column.

In [ ]:
df2 = df2[['Summer gold', 'Winter gold']]
df2

In [ ]:
df2['Winter gold'].sum()

We can also select by row and column simultaneously to create a new data frame

In [ ]:
df[["Winter gold", "Winter silver", "Winter bronze"]].iloc[10:15]

Since slicing by row is so common, pandas lets you skip using iloc and simply provide a range of indices

In [ ]:
df[["Winter gold", "Winter silver", "Winter bronze"]][10:15]

Selection can also be done by row labels using loc with a single label or list of labels

In [ ]:
df[["Winter gold", "Winter silver", "Winter bronze"]].loc[['Barbados', 'Belarus', 'Belgium']]

## Missing values

Observation data often has missing values, which can lead to problems with their analysis. Fortunately, pandas provides fillna and interpolate methods for filling in these missing values (data imputation)

Let's explore this using a small data set containing temperatures in US cities over the course of a week.

In [ ]:
# Missing values are dislayed as NaNs (Not a Number)
df3 = pd.read_csv('city temps.csv', index_col=0, skiprows=0)
df3

In [ ]:
# The simplest method is to replace missing values with a fixed value
df3.fillna(value=70)

In [ ]:
# Forward filling using the last valid value to fill missing value
# Note that in this case we used axis=1 so that we propogate across
# the rows rather than the values

df3.fillna(method="ffill", axis=1)

In [ ]:
# Back filling using the NEXT valid value to fill missing value
# Note that in this case we used axis=1 so that we propogate across
# the rows rather than the values

df3.fillna(method="bfill", axis=1)

In [ ]:
# As of version 0.17.0, pandas provides an interpolate function that fills missing values
# By default, we get a linear interpolation, but a number of other options are available
# (quadratic, cubic, polynomial, etc.)

df3.interpolate(method="linear", axis=1)

## Reading from other file formats

Until now, we've been working with csv files, but pandas can handle many other formats including json, html, excel and HDF. This is extremely useful since we don't need to create csv files from richer data formats. We show an example below where we read two sheets from an Excel file

In [ ]:
df4 = pd.read_excel('city temps spreadsheet.xlsx', index_col=0, skiprows=0, sheet_name='set1')
df4

In [ ]:
df5 = pd.read_excel('city temps spreadsheet.xlsx', index_col=0, skiprows=0, sheet_name='set2')
df5

## From numpy array to data frame

Many Python packages can be used with numpy arrays or pandas data frames, but it's often easier to work with the latter. Pandas provides a simple way to convert an array to a data frame.

In [ ]:
import numpy as np
a = np.random.rand(10,3)
a

In [ ]:
df6 = pd.DataFrame(a, columns=['feature 1', 'feature 2', 'feature 3'])
df6